In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS187
## Lab 1-4 – Discriminative methods for classification

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* `numpy.exp`

## Preparation – Loading packages and data

In [1]:
import os
import json

import math
from math import log2

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('tableau-colorblind10')

import numpy as np
from pprint import pprint

# Otter grader which we use for grading does not support
# !command, so we need to use shell(command) instead
# to run shell commands
def shell(str):
    file = os.popen(str)
    result = file.read()
    print (result)
    if file.close () is not None:
        print ('failed')

In [2]:
# Read the Federalist data from the json file
shell('wget -nv -N -P data https://github.com/nlp-course/data/raw/master/Federalist/federalist_data.json')
with open('data/federalist_data.json', 'r') as fin:
    dataset = json.load(fin)

In [3]:
# As before, we extract the papers by either of Madison and Hamilton to serve as training data.
training = list(filter(lambda ex: ex['authors'] in ['Madison', 'Hamilton'],
                       dataset))

## Logistic regression

You've read about logistic regression, a method for classification that is discriminative rather than generative. Like the generative Naive Bayes method, each example is characterized by a vector of features (the counts of word types for a text, say, as for the _Federalist_ example we've been using). In logistic regression, each such feature is assigned a weight, and the score for an example is given by weighting each feature value by its weight and summing the result; that is, the score is the dot product of the feature vector and the weight vector. Let's take an example, one of the Federalist papers. We'll extract from the training data the counts for the first example in the training set. That's the feature vector for this example.

In [4]:
training0_counts = training[0]['counts']

Suppose the weights for the features are as given by the following vector:

In [5]:
weights = [-.1, .2, .3, -1]

What would the weighted sum of the counts with these weights be? Feel free to use `np.dot` to take the dot product for you.
<!--
BEGIN QUESTION
name: wtd_sum
-->

In [6]:
wtd_sum = ...

In [ ]:
grader.check("wtd_sum")

<!-- BEGIN QUESTION -->

**Question:** What is the range of possible values that such a weighted sum can take on?
<!--
BEGIN QUESTION
name: open_response_1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

In order to have a standard way of comparing these numbers, it helps to be able to place them on a fixed scale, from 0 to 1, say. This way, they can be interpreted as probabilities. We use the _logistic function_ ($\sigma$) to carry out this conversion:

$$ \sigma(x) = \frac{1}{1 + e^{-k x}}$$

In addition to its argument $x$, the function takes an additional parameter $k$, which we will explore shortly.

Define a function `sigma` implementing the logistic function. (We've established a default value for `k` of 1 in the header line below.)
<!--
BEGIN QUESTION
name: sigma
-->

In [9]:
def sigma(x, k=1):
    ...

In [ ]:
grader.check("sigma")

To get a sense of the logistic function, we graph it for several values of $k$.

In [12]:
def sigma_plot():
    x = np.linspace(-2, 2, 100)
    fig, ax = plt.subplots()
    for k in [0, 1, 2, 10]:
        ax.plot(x, sigma(x, k), label = f"k = {k}")
    plt.title("Logistic functions")
    plt.legend()
    
sigma_plot()

<!-- BEGIN QUESTION -->

**Question:** What does the $k$ parameter do to the logistic function?
<!--
BEGIN QUESTION
manual: true
name: open_response_2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



The logistic function, when applied to the weighted average above is quite close to 1.

In [13]:
sigma(wtd_sum)

In the _Federalist Papers_ classification problem, there are only two classes, so we can use this single value to determine the classification. For a feature vector $\mathbf{x}$, we'll take the model to predict the probability of the author being Hamilton (say), as

$$\Prob(\mathrm{Hamilton} \given \mathbf{x}) = \sigma(\mathbf{w} \cdot \mathbf{x})$$

Therefore, 

$$\Prob(\mathrm{Madison} \given \mathbf{x}) = 1 - \sigma(\mathbf{w} \cdot \mathbf{x})$$

since there are only two classes.

> When there are more than two classes, we'd use a generalization of the sigmoid function, called [softmax](https://en.wikipedia.org/wiki/Softmax_function).

Define a function `predict_lr` that calculates the probability of Hamilton being the author of an example text, given a weight vector and the feature vector for the example text.
<!--
BEGIN QUESTION
name: predict_lr
-->

In [14]:
def predict_lr(weights, features):
    ...

## Using a logistic regression model to predict Federalist authorship

Consider the following two training examples (examples 0 and 9) from the Federalist dataset:

In [15]:
for example in [0, 9]:
    pprint(training[example])

As above, a logistic regression model is defined by a vector of weights $\mathbf{w}$, like these:

In [16]:
weights

Calculate the predicted Hamilton probabilities for the two examples (examples 0 and 9) above.
<!--
BEGIN QUESTION
name: prob_hamilton_examples
-->

In [17]:
prob_hamilton_example0 = ...
prob_hamilton_example9 = ...

In [ ]:
grader.check("prob_hamilton_examples")

In [20]:
print(f"example 0 prob: {prob_hamilton_example0:.3f}\n"
      f"example 9 prob: {prob_hamilton_example9:.3f}")

<!-- BEGIN QUESTION -->

**Question:** What does this model predict about the two training examples? Is it correct?
<!--
BEGIN QUESTION
name: open_response_3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Training a logistic regression model

Of course, this is just one of a gazillion models, since we could have set the weights in all kinds of ways. How should we come up with a _good_ model, a good setting of the weights? Ideally, we'd try to find a set of weights that predicts all of the training data well. This is the problem of _training_ a logistic regression model. Let's try another set of weights:

In [21]:
weights_new = [0.1, 0.2, 0.3, -5]

Calculate the probabilities generated by the model for these weights.
<!--
BEGIN QUESTION
name: prob_hamilton_examples_new
-->

In [22]:
prob_hamilton_example0_new = ...
prob_hamilton_example9_new = ...

In [ ]:
grader.check("prob_hamilton_examples_new")

<!-- BEGIN QUESTION -->

**Question:**  Is this a better model or a worse model, at least as far as the two sample examples go? 
<!--
BEGIN QUESTION
name: open_response_4
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



An ideal model would give a probability of 1 to the Hamilton examples and a 0 to the Madison examples.
For the two sample examples, you'll have noticed that the new weights generate not 1 and 0, respectively, but numbers quite a bit closer to 1 and 0.

We could continue trying different weight values to try to improve the performance of the model on these training examples (and others) by trial and error, but a more systematic method is needed. We define a _loss function_, which specifies how bad a model is, and try to minimize the loss function by _stochastic gradient descent_.

We'll do a few steps of the process here by hand. It's sufficiently tedious that it's far better to deploy computers on the task.

## Cross-entropy loss function

We'll use the cross-entropy loss function. For an example $i$, we'll use $\mathbf{x}^{(i)}$ for the feature vector, and $y^{(i)}$ for the actual (gold) label (1 or 0). The predicted label will be as per the logistic regression model $\sigma(\mathbf{w} \cdot \mathbf{x}^{(i)})$, which we will call $\hat{y}^{(i)}$.

The cross-entropy loss for example $i$ as per a model $\mathbf{w}$ is
$$L_{CE}(\mathbf{w}) = -(y^{(i)} \log \hat{y}^{(i)} + (1-y^{(i)}) \log (1-\hat{y}^{(i)}))$$

We define a function to compute the cross-entropy loss for a particular model (weight vector), example (feature vector), and gold label:

In [25]:
def loss(weights, features, correct):
    """Returns the cross-entropy loss for a weight vector, a feature
       vector and a gold label"""
    y_hat = predict_lr(weights, features)
    return -(correct * log2(y_hat)
             + (1 - correct) * log2(1 - y_hat))

Use the `loss` function to determine the cross-entropy loss for example 0 for the original model that we used (`weights`), and for the new model (`weights_new`).
<!--
BEGIN QUESTION
name: loss_example0_old_and_new
-->

In [26]:
loss_example0_old = ...
loss_example0_new = ...

In [ ]:
grader.check("loss_example0_old_and_new")

In [29]:
print(f"Old model loss: {loss_example0_old:.3f}\n"
      f"New model loss: {loss_example0_new:.3f}")

<!-- BEGIN QUESTION -->

**Question:** Which of the models is better (at least on this example)?
<!--
BEGIN QUESTION
name: open_response_5
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Gradient of the loss function

We want to find the weights that minimize the loss function. We use gradient descent:

1. Find the gradient of the loss function, the direction in which it is increasing fastest.
2. Take a step in the opposite direction.
3. Repeat.

For cross-entropy loss, recall that the partial derivative of the loss function with respect to a single weight $w_j$ is

$$ \frac{\partial L_{CE}(\mathbf{w})}{\partial w_j} = (\hat{y} - y) x_j $$

The gradient combines these partial derivatives for all of the weights.

$$ \nabla L_{CE}(\mathbf{w}) = \left[ \begin{array}{c}
    \frac{\partial L_{CE}(\mathbf{w})}{\partial w_1}\\
    \frac{\partial L_{CE}(\mathbf{w})}{\partial w_2}\\
    \vdots \\
    \frac{\partial L_{CE}(\mathbf{w})}{\partial w_m}
    \end{array} \right] $$

Let's work out an example, using example 0.

The counts for example 0 are

In [30]:
training[0]['counts']

and the original weights, recall, were

In [31]:
weights

What is the gradient vector for these weights and training example?
<!--
BEGIN QUESTION
name: grad_vector
-->

In [32]:
...
grad_vector = ...
grad_vector

In [ ]:
grader.check("grad_vector")

Step 2 is to adjust the weights in the _opposite_ direction of the gradient.
In this case, we compute the new weight vector $w'$ by adding to the weight vector a fraction of the negative gradient:

$$ \mathbf{w}' = \mathbf{w} - \eta \nabla L_{CE}(\mathbf{w}) $$

Here, $\eta$ is the _learning rate_. The larger $\eta$ is, the more we move in each step, but if $\eta$ is too large we risk overshooting. We'll use a learning rate of 0.1 for now. (Setting good learning rates is one aspect of the black arts of machine learning.)

In [35]:
learning_rate = 0.1

Calculate the new weight vector.
<!--
BEGIN QUESTION
name: weights_updated
-->

In [36]:
weights_updated = ...

In [ ]:
grader.check("weights_updated")

In [39]:
pprint(weights_updated)

How do these weights perform on the training example we've been using? Let's see.

In [40]:
loss_example0_updated = loss(weights_updated, training[0]['counts'], 1)

In [41]:
print(f"Old model loss: {loss_example0_old:.3f}\n"
      f"New model loss: {loss_example0_new:.3f}\n"
      f"Updated model loss: {loss_example0_updated:.3f}")

If you did this all right, the loss for the updated model, which was generated by taking a single step opposite the gradient from the old model is not only better than the old model, but better than the new model we used above as well. 

What about the loss on the other example we've been using (example 9)? Calculate the loss for example 9 with the old model, the new model, and the updated model:
<!--
BEGIN QUESTION
name: weights_updated_9
-->

In [42]:
loss_example9_old = loss(weights, training[9]['counts'], 0) # Solution
loss_example9_new = loss(weights_new, training[9]['counts'], 0) # Solution
loss_example9_updated = loss(weights_updated, training[9]['counts'], 0) # Solution

In [ ]:
grader.check("weights_updated_9")

In [45]:
print(f"Old model loss: {loss_example9_old:.3f}\n"
      f"New model loss: {loss_example9_new:.3f}\n"
      f"Updated model loss: {loss_example9_updated:.3f}")

<!-- BEGIN QUESTION -->

**Question:** Did the update to the model improve its performance on example 9 or make it worse?
<!--
BEGIN QUESTION
name: open_response_6
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



Step 3 is to repeat the process for this and other training examples. We could recalculate the gradient and take another step to improve further, and take steps to improve the other training examples, and so on and so forth, eventually converging on a model that works well over the entire training set. But doing so manually in this way is too tedious. We need to be able to do these kinds of computations at scale. Fortunately, we'll soon be turning to packages that allow specifying these larger-scale computations.

# End of lab 1-4

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()